# Product Dataset Processing

In [1]:
# pip install pymorphy2

In [26]:
import os
import string
import annoy
import json

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [4]:
products = pd.read_csv('../input/products/ProductsDataset.csv')
products

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...
...,...,...,...,...,...,...,...
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9.0,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9.0,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9.0,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...
35546,Новая золотая ветровка,Женская ветровка размер 44-46. Цвет приглушённ...,5bd6c8fb2138bbc55745362c,9.0,903,"{'zhenskaya_odezhda_razmer': '44-46 (М)', 'zhe...",http://cache3.youla.io/files/images/360_360/5b...


In [5]:
products[products['product_id'].isna()]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
4649,Кепка человек паук размер 52-54,Кепка человек паук,NaN,NaN,NaN,NaN,NaN
4650,размер 52-54,NaN,NaN,NaN,NaN,NaN,NaN
4651,взади сетка,NaN,NaN,NaN,NaN,NaN,NaN
4652,----------,NaN,NaN,NaN,NaN,NaN,NaN
4653,Могу выслать в любой город Р.Ф.,NaN,NaN,NaN,NaN,NaN,NaN
26827,"Лампа галогенная GU5.3,GU10 MR16 20,35,50,75Вт",camelion,NaN,NaN,NaN,NaN,NaN
26828,117шт*15р,NaN,NaN,NaN,NaN,NaN,NaN
27016,Кепка Minnie Mouse C прямым козырьком размер 5...,Кепка Minnie Mouse,NaN,NaN,NaN,NaN,NaN
27017,C прямым козырьком,NaN,NaN,NaN,NaN,NaN,NaN
27018,Размер 50-54,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Delete rows without product_id
products.dropna(subset=['product_id'], inplace=True)

In [7]:
products = products[['title', 'descrirption', 'product_id']]
products.rename(columns={'descrirption': 'description'}, inplace=True)
products['description'].fillna('', inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [8]:
products['text'] = products['title'] + ' ' + products['description']
products = products.astype('string')
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35536 entries, 0 to 35547
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        35536 non-null  string
 1   description  35536 non-null  string
 2   product_id   35536 non-null  string
 3   text         35536 non-null  string
dtypes: string(4)
memory usage: 1.4 MB


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

# The function for text processing
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [14]:
products['text'] = products['text'].apply(preprocess_txt)

In [15]:
products.reset_index(drop=True, inplace=True)

In [16]:
products.sample(5)

,title,description,product_id,text
34810,Зимнее пальто с натуральным мехом,Зимнее пальто в черном цвете в наличии . Длина...,5bb11d8c65bcf18c5501f5fa,"[зимний, пальто, натуральный, мех, зимний, пал..."
4912,❗Джемпер новый,"Джемпер новый из хлопка, 🔸цвет серый меланж, ...",5b9be467132ca525e70ad210,"[❗джемпер, новый, джемпер, новый, хлопок, 🔸цве..."
10116,Кеды детские (бу),"Продам детские кеды(бу) 1,2 фото размер 30,сос...",5ad85bb1f8efdc29c37f15d2,"[кед, детский, бу, продать, детский, кедыба, 1..."
25618,Детские кеды Gymboree!!!,Очень милые и симпатичные кедики американской ...,5c0cc83ade8854c9b9244937,"[детский, кед, gymboree, милый, симпатичный, к..."
14894,Сарафан,Новый!,5b3f9e202c593e0f471fd962,"[сарафан, новый]"


In [17]:
products.to_csv('prod_proc.csv')

In [18]:
prod_list = []
for item in products['text'].values:
    prod_list.append(item)

In [19]:
# Apply Word2Vec
prod_list = [i for i in prod_list if len(i) > 2]
model = Word2Vec(sentences=prod_list, vector_size=100, min_count=1, window=5)
model.save("w2v_model_prod.model")

In [20]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec_prod.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
#word_vectors = KeyedVectors.load("../input/models/word2vec.wordvectors", mmap='r')

In [21]:
keys = word_vectors.key_to_index # dictionary of all words in Word2Vec model

In [22]:
len(keys)

50410

In [24]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}

for i in range(products.shape[0]):
    reply = str(products['product_id'].loc[i]) + " " + str(products['title'].loc[i])
    index_map[i] = reply
    question = products['title'].loc[i]
    question = preprocess_txt(question)
    vector = np.zeros(100)
    n_w2v = 0
    for word in question:
        if word in keys.keys():
            vector += word_vectors[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index.add_item(i, vector)

index.build(10)
index.save('index_prod.ann')

True

In [27]:
with open('index_map_prod.json', 'w') as fp:
    json.dump(index_map, fp)